In [1]:
!pip install plot-keras-history

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for plot-keras-history: filename=plot_keras_history-1.1.39-py3-none-any.whl size=10667 sha256=39a3756f83833ff75546517b59fa618dfc0f9b02aaf2de34605b61893ab744e9
  Stored in directory: /root/.cache/pip/wheels/03/3e/64/c18638ae75cbffd7c5aaf2abef32fa6050c060b4fde0c0bef6
  Created wheel for sanitize_ml_labels: filename=sanitize_ml_labels-1.1.4-py3-none-any.whl size=324285 sha256=ab356054d6bd5e5573983d8e1a18867be36ca5e457e0007925c5e7bc7f906bf0
  Stored in directory: /root/.cache/pip/wheels/5c/64/72/7d3cf6d961a77d1bdd8486046f45c1eb4a3516b570f7258e29
  Created wheel for compress-json: filename=compress_json-1.1.1-py3-none-any.whl size=6598 sha256=d4064072e74c9c99a6df137f9446194421bc4954d05931c7e3cb53425c3c04a9
  Stored in directory: /root/.cache/pip/wheels/be/d3/03/626b5d7f1d734

In [2]:
from __future__ import print_function, division

# Thư viện hệ thống
import os
import re
import random
import math

# Khoa học dữ liệu
import numpy as np
import pandas as pd

# TensorFlow và Keras (dùng tf.keras là chuẩn nhất trong TensorFlow 2.x)
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Embedding, Conv1D, Conv2D, GlobalMaxPooling1D, GlobalMaxPooling2D,
    Reshape, Dense, Dropout, Flatten, MaxPooling1D, Input, Concatenate,
    LSTM, Bidirectional
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers # Import the layers module

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier  # lightgbm chuẩn

# PyTorch (nếu dùng trong phần riêng biệt)
import torch
from torch import nn

# Visualize training history
import matplotlib.pyplot as plt
# Nếu không có sẵn, cài đặt bằng: pip install plot-keras-history
from plot_keras_history import plot_history

# Nếu cần sử dụng linalg
from numpy import linalg as la


# pd.set_option('display.max_columns',None)

# np.set_printoptions(threshold=np.inf)

"""Import Data"""

def parse_stream(f, comment=b'#'):
    name = None
    sequence = []
    for line in f:
        if line.startswith(comment):
            continue
        line = line.strip()
        if line.startswith(b'>'):
            if name is not None:
                yield name, b''.join(sequence)
            name = line[1:]
            sequence = []
        else:
            sequence.append(line.upper())
    if name is not None:
        yield name, b''.join(sequence)

def fasta2csv(inFasta):
    FastaRead=pd.read_csv(inFasta,header=None)
    print(FastaRead.shape)
    print(FastaRead.head())
    seqNum=int(FastaRead.shape[0]/2)
    csvFile=open("testFasta.csv","w")
    csvFile.write("PID,Seq\n")

    #print("Lines:",FastaRead.shape)
    #print("Seq Num:",seqNum)
    for i in range(seqNum):
      csvFile.write(str(FastaRead.iloc[2*i,0])+","+str(FastaRead.iloc[2*i+1,0])+"\n")


    csvFile.close()
    TrainSeqLabel=pd.read_csv("testFasta.csv",header=0)
    path="testFasta.csv"
    if os.path.exists(path):

        os.remove(path)

    return TrainSeqLabel

inFastaTrain="/content/Train_set.fasta"
inFastaTest="/content/Independent_test_converted.fasta"

mainTrain = fasta2csv(inFastaTrain)

mainTrain.head()

mainTest = fasta2csv(inFastaTest)

mainTest.head()

"""process y"""

i=0
mainTrain["Tags"]=mainTrain["Seq"]
for i, pid in enumerate(mainTrain["PID"]):
  mainTrain.loc[i, "Tags"] = int(pid[-1])

for i, pid in enumerate(mainTest["PID"]):
  mainTest.loc[i, "Tags"] = int(pid[-1])


ACP_y_train = mainTrain["Tags"].values
ACP_y_test = mainTest["Tags"].values

ACP_y_train_ = [np.array(i) for i in ACP_y_train]
ACP_y_test_ = [np.array(i) for i in ACP_y_test]

ACP_y_train_ = np.array(ACP_y_train_)
ACP_y_test_ = np.array(ACP_y_test_)

"""protein_seq_dict=>x_train"""

x_train = {}
protein_index = 1
for line in mainTrain["Seq"]:
  x_train[protein_index] = line
  protein_index = protein_index + 1

x_test = {}
protein_index = 1
for line in mainTest["Seq"]:
  x_test[protein_index] = line
  protein_index = protein_index + 1

#The longest length of train protein is 50
maxlen_train = max(len(x) for x in x_train.values())
print(maxlen_train)

#The longest length of test protein is 50
maxlen_test = max(len(x) for x in x_test.values())
print(maxlen_test)

maxlen = max(maxlen_train,maxlen_test)
maxlen

"""Add features"""

#Convert amino acids to vectors
def OE(seq_temp):
    seq = seq_temp
    chars = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y']
    fea = []
    tem_vec =[]
    #k = 6
    for i in range(len(seq)):
        if seq[i] =='A':
            tem_vec = [1]
        elif seq[i]=='C':
            tem_vec = [2]
        elif seq[i]=='D':
            tem_vec = [3]
        elif seq[i]=='E' or seq[i]=='U':
            tem_vec = [4]
        elif seq[i]=='F':
            tem_vec = [5]
        elif seq[i]=='G':
            tem_vec = [6]
        elif seq[i]=='H':
            tem_vec = [7]
        elif seq[i]=='I':
            tem_vec = [8]
        elif seq[i]=='K':
            tem_vec = [9]
        elif seq[i]=='L':
            tem_vec = [10]
        elif seq[i]=='M' or seq[i]=='O':
            tem_vec = [11]
        elif seq[i]=='N':
            tem_vec = [12]
        elif seq[i]=='P':
            tem_vec = [13]
        elif seq[i]=='Q':
            tem_vec = [14]
        elif seq[i]=='R':
            tem_vec = [15]
        elif seq[i]=='S':
            tem_vec = [16]
        elif seq[i]=='T':
            tem_vec = [17]
        elif seq[i]=='V':
            tem_vec = [18]
        elif seq[i]=='W':
            tem_vec = [19]
        elif seq[i]=='X' or seq[i]=='B' or seq[i]=='Z':
            tem_vec = [20]
        elif seq[i]=='Y':
            tem_vec = [21]
        #fea = fea + tem_vec +[i]
        fea.append(tem_vec)
    return fea

x_train_OE = []
for i in x_train:
  OE_feature = OE(x_train[i])
  x_train_OE.append(OE_feature)
  #print(protein_seq_dict[i])

x_test_OE = []
for i in x_test:
  OE_feature = OE(x_test[i])
  x_test_OE.append(OE_feature)

"""AAC"""

handcraft_AAC_train = [[0] * 21 for _ in range(len(x_train_OE))]
for row in range(len(x_train_OE)):
  seq = x_train_OE[row]
  for i in seq:
    col = i[0]-1
    handcraft_AAC_train[row][col] += 1/len(seq)
hc_AAC_train = np.array(handcraft_AAC_train)

handcraft_AAC_test = [[0] * 21 for _ in range(len(x_test_OE))]
for row in range(len(x_test_OE)):
  seq = x_test_OE[row]
  for i in seq:
    col = i[0]-1
    handcraft_AAC_test[row][col] += 1/len(seq)
hc_AAC_test = np.array(handcraft_AAC_test)

"""DPC"""

comb = []
for i in range(1,22):
  for j in range(i,22):
    comb.append([i,j])
comb_index = {}
for i in range(len(comb)):
  comb_index[tuple(comb[i])] = i
handcraft_DPC_train = [[0] * len(comb) for _ in range(len(x_train_OE))]
for row in range(len(x_train_OE)):
  seq = x_train_OE[row]
  for i in range(len(seq)-1):
    a = sorted([seq[i][0],seq[i+1][0]])
    index = comb_index[tuple(a)]
    handcraft_DPC_train[row][index] += 1/(len(seq)-1)
hc_DPC_train = np.array(handcraft_DPC_train)

handcraft_DPC_test = [[0] * len(comb) for _ in range(len(x_test_OE))]
for row in range(len(x_test_OE)):
  seq = x_test_OE[row]
  for i in range(len(seq)-1):
    a = sorted([seq[i][0],seq[i+1][0]])
    index = comb_index[tuple(a)]
    handcraft_DPC_test[row][index] += 1/(len(seq)-1)
hc_DPC_test = np.array(handcraft_DPC_test)

"""CKSAAGP"""

import re
import sys
def readFasta(file):
    if os.path.exists(file) == False:
        print('Error: "' + file + '" dOEs not exist.')
        sys.exit(1)

    with open(file) as f:
        records = f.read()

    if re.search('>', records) == None:
        print('The input file seems not in fasta format.')
        sys.exit(1)

    records = records.split('>')[1:]
    myFasta = []
    for fasta in records:
        array = fasta.split('\n')
        name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
        myFasta.append([name, sequence])

    return myFasta

def generateGroupPairs(groupKey):
    gPair = {}
    for key1 in groupKey:
        for key2 in groupKey:
            gPair[key1+'.'+key2] = 0
    return gPair


def CKSAAGP(fastas, gap = 5, **kw):

    group = {
        'alphaticr': 'GAVLMI',
        'aromatic': 'FYW',
        'postivecharger': 'KRH',
        'negativecharger': 'DE',
        'uncharger': 'STCPNQ'
    }

    AA = 'ARNDCQEGHILKMFPSTWYV'

    groupKey = group.keys()

    index = {}
    for key in groupKey:
        for aa in group[key]:
            index[aa] = key

    gPairIndex = []
    for key1 in groupKey:
        for key2 in groupKey:
            gPairIndex.append(key1+'.'+key2)

    encodings = []
    header = ['#']
    for g in range(gap + 1):
        for p in gPairIndex:
            header.append(p+'.gap'+str(g))
    encodings.append(header)

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = [name]
        for g in range(gap + 1):
            gPair = generateGroupPairs(groupKey)
            sum = 0
            for p1 in range(len(sequence)):
                p2 = p1 + g + 1
                if p2 < len(sequence) and sequence[p1] in AA and sequence[p2] in AA:
                    gPair[index[sequence[p1]]+'.'+index[sequence[p2]]] = gPair[index[sequence[p1]]+'.'+index[sequence[p2]]] + 1
                    sum = sum + 1

            if sum == 0:
                for gp in gPairIndex:
                    code.append(0)
            else:
                for gp in gPairIndex:
                    code.append(gPair[gp] / sum)

        encodings.append(code)

    return encodings

handcraft_CKSAAGP_train = CKSAAGP(readFasta(inFastaTrain))
handcraft_CKS_train = []
for i in range(1,len(handcraft_CKSAAGP_train)):
  handcraft_CKS_train.append(handcraft_CKSAAGP_train[i][1:])
hc_CKS_train = np.array(handcraft_CKS_train)

handcraft_CKSAAGP_test = CKSAAGP(readFasta(inFastaTest))
handcraft_CKS_test = []
for i in range(1,len(handcraft_CKSAAGP_test)):
  handcraft_CKS_test.append(handcraft_CKSAAGP_test[i][1:])
hc_CKS_test = np.array(handcraft_CKS_test)

"""K-mer"""

def TransDict_from_list(groups):
    transDict = dict()
    tar_list = ['0', '1', '2', '3', '4', '5', '6']
    result = {}
    index = 0
    for group in groups:
        g_members = sorted(group)  # Alphabetically sorted list
        for c in g_members:
            # print('c' + str(c))
            # print('g_members[0]' + str(g_members[0]))
            result[c] = str(tar_list[index])  # K:V map, use group's first letter as represent.
        index = index + 1
    return result

def get_3_protein_trids():
    nucle_com = []
    chars = ['0', '1', '2', '3', '4', '5', '6']
    base = len(chars)
    end = len(chars) ** 3
    for i in range(0, end):
        n = i
        ch0 = chars[n % base]
        n = n / base
        ch1 = chars[int(n % base)]
        n = n / base
        ch2 = chars[int(n % base)]
        nucle_com.append(ch0 + ch1 + ch2)
    return nucle_com

def translate_sequence(seq, TranslationDict):
    '''
    Given (seq) - a string/sequence to translate,
    Translates into a reduced alphabet, using a translation dict provided
    by the TransDict_from_list() method.
    Returns the string/sequence in the new, reduced alphabet.
    Remember - in Python string are immutable..

    '''
    import string
    from_list = []
    to_list = []
    for k, v in TranslationDict.items():
        from_list.append(k)
        to_list.append(v)
    # TRANS_seq = seq.translate(str.maketrans(zip(from_list,to_list)))
    TRANS_seq = seq.translate(str.maketrans(str(from_list), str(to_list)))
    # TRANS_seq = maketrans( TranslationDict, seq)
    return TRANS_seq

def get_4_nucleotide_composition(tris, seq, pythoncount=True):
    seq_len = len(seq)
    tri_feature = [0] * len(tris)
    k = len(tris[0])
    note_feature = [[0 for cols in range(len(seq) - k + 1)] for rows in range(len(tris))]
    if pythoncount:
        for val in tris:
            num = seq.count(val)
            tri_feature.append(float(num) / seq_len)
    else:
        # tmp_fea = [0] * len(tris)
        for x in range(len(seq) + 1 - k):
            kmer = seq[x:x + k]
            if kmer in tris:
                ind = tris.index(kmer)
                # tmp_fea[ind] = tmp_fea[ind] + 1
                note_feature[ind][x] = note_feature[ind][x] + 1
        # tri_feature = [float(val)/seq_len for val in tmp_fea]    #tri_feature type:list len:256
        u, s, v = la.svd(note_feature)
        for i in range(len(s)):
            tri_feature = tri_feature + u[i] * s[i] / seq_len
        # print tri_feature
        # pdb.set_trace()

    return tri_feature

def prepare_feature_kmer(protein_seq_dict):
    label = []
    interaction_pair = {}
    RNA_seq_dict = {}

    groups = ['AGV', 'ILFP', 'YMTS', 'HNQW', 'RK', 'DE', 'C']
    group_dict = TransDict_from_list(groups)
    protein_tris = get_3_protein_trids()

    kmer = []

    for i in protein_seq_dict:
        protein_seq = translate_sequence(protein_seq_dict[i], group_dict)
        protein_tri_fea = get_4_nucleotide_composition(protein_tris, protein_seq, pythoncount =False)

        kmer.append(protein_tri_fea)

    return np.array(kmer)

kmer_train=prepare_feature_kmer(x_train)
kmer_test=prepare_feature_kmer(x_test)

"""4 Features:4748"""

hc_train = np.c_[hc_AAC_train,hc_DPC_train,hc_CKS_train,kmer_train]
hc_train.shape

hc_test = np.c_[hc_AAC_test,hc_DPC_test,hc_CKS_test,kmer_test]
hc_test.shape

X=hc_train
print(X.shape)

"""four feature augment"""

fea_num=745

def oversamp_pos(X_result, p):
  delta = 0.05
  add_num = int(len(X_result)*p)
  X_add_all = []

  for i in range(add_num):
    idx_ram = random.randint(0,X_result.shape[0]-1)
    X_sel = X_result[idx_ram,:]
    value = np.random.uniform(0, 1, (1, fea_num))
#    value = np.concatenate((value1, value2),axis = 1)
#    value = np.random.normal(0,1)
#    value = np.random.poisson(6, size=(1,402))
#    value = np.random.exponential(10, size=(1,402))
    add_value = value*delta*X_sel

    X_add = X_sel + add_value
    X_add = np.squeeze(X_add)
    X_add_all.append(X_add)

  X_add_all = np.array(X_add_all)
  return X_add_all#,label_add

def oversamp_neg(X_result, p):
  delta = 0.05
  add_num = int(len(X_result)*p)
  X_add_all = []

  for i in range(add_num):
    idx_ram = random.randint(0,X_result.shape[0]-1)
    X_sel = X_result[idx_ram,:]
    value = np.random.uniform(0, 1, (1, fea_num))
#    value = np.concatenate((value1, value2),axis = 1)
#    value = np.random.normal(0,1)
#    value = np.random.poisson(6, size=(1,402))
#    value = np.random.exponential(10, size=(1,402))
    add_value = value*delta*X_sel
    X_add = X_sel + add_value
    X_add = np.squeeze(X_add)
    X_add_all.append(X_add)

  X_add_all = np.array(X_add_all)
  return X_add_all#,label_add

def ACP_DL():
    np.random.seed(0)
    random.seed(0)
    augtimes = 1

    train = X
    train_label = ACP_y_train_

    # augment the train data
    idx_pos = (train_label == 1)
    idx_neg = (train_label == 0)
    X_pos = train[idx_pos,:]
    X_neg = train[idx_neg,:]
    X_pos_add = oversamp_pos(X_pos, augtimes)
    X_neg_add = oversamp_neg(X_neg, augtimes)

    X_origin = np.concatenate((X_pos,X_neg))
    X_new = np.concatenate((X_pos_add,X_neg_add))
    X_aug = np.concatenate((X_origin,X_new))

    label_aug = np.hstack([train_label]*2)

    return X_aug,label_aug

X_aug,label_aug=ACP_DL()

X_aug.shape

"""add position info in OE"""

#Convert amino acids to vectors
def OE_position(seq_temp):
    seq = seq_temp
    chars = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y']
    fea = []
    tem_vec =[]
    #k = 6
    for i in range(len(seq)):
        if seq[i] =='A':
            tem_vec = [0*50+i]
        elif seq[i]=='C':
            tem_vec = [1*50+i]
        elif seq[i]=='D':
            tem_vec = [2*50+i]
        elif seq[i]=='E' or seq[i]=='U':
            tem_vec = [3*50+i]
        elif seq[i]=='F':
            tem_vec = [4*50+i]
        elif seq[i]=='G':
            tem_vec = [5*50+i]
        elif seq[i]=='H':
            tem_vec = [6*50+i]
        elif seq[i]=='I':
            tem_vec = [7*50+i]
        elif seq[i]=='K':
            tem_vec = [8*50+i]
        elif seq[i]=='L':
            tem_vec = [9*50+i]
        elif seq[i]=='M' or seq[i]=='O':
            tem_vec = [10*50+i]
        elif seq[i]=='N':
            tem_vec = [11*50+i]
        elif seq[i]=='P':
            tem_vec = [12*50+i]
        elif seq[i]=='Q':
            tem_vec = [13*50+i]
        elif seq[i]=='R':
            tem_vec = [14*50+i]
        elif seq[i]=='S':
            tem_vec = [15*50+i]
        elif seq[i]=='T':
            tem_vec = [16*50+i]
        elif seq[i]=='V':
            tem_vec = [17*50+i]
        elif seq[i]=='W':
            tem_vec = [18*50+i]
        elif seq[i]=='X' or seq[i]=='B' or seq[i]=='Z':
            tem_vec = [19*50+i]
        elif seq[i]=='Y':
            tem_vec = [20*50+i]
        #tem_vec = tem_vec +[i]
        fea.append(tem_vec)
        #fea.append(tem_vec)
    return fea

x_train_OE = []
for i in x_train:
  OE_feature = OE_position(x_train[i])
  x_train_OE.append(OE_feature)

x_test_OE = []
for i in x_test:
  OE_feature = OE_position(x_test[i])
  x_test_OE.append(OE_feature)

"""The longest protein sequence has 50 amino acids, and each amino acid has 21 dimensions. Complete the short sequences."""

x_train_OE = pad_sequences(x_train_OE, maxlen=maxlen, padding='post',
                           truncating='post', value=0, dtype='int32')
x_test_OE  = pad_sequences(x_test_OE,  maxlen=maxlen, padding='post',
                           truncating='post', value=0, dtype='int32')

"""copy OE"""

x_train_OE_aug = np.vstack([x_train_OE]*2)

len(x_train_OE_aug)

"""Select Features"""

def RF_SelectFeatures(X,y,i):#RadomForest Feature Selection Method

    model = RandomForestClassifier(n_estimators=888,min_samples_leaf=10,random_state=2020,n_jobs=8)
    model.fit(X, y)
    importantFeatures = model.feature_importances_
    K = importantFeatures.argsort()[::-1][:i]
    RF_ALL_K=X[:,K]

    return  RF_ALL_K

def LGBM_SelectFeatures(X,y,i):#Light Gradient Boosting Machine Feature Selection

    model = LGBMClassifier(num_leaves=32,n_estimators=888,max_depth=12,learning_rate=0.16,min_child_samples=50,random_state=2020,n_jobs=8)
    model.fit(X, y)
    importantFeatures = model.feature_importances_
    K = importantFeatures.argsort()[::-1][:i]
    LGB_ALL_K=X[:,K]

    return  LGB_ALL_K

"""Attention"""


import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.layers import Layer

from tensorflow.keras import backend as K, initializers, regularizers, constraints
from tensorflow.keras.layers import Layer

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
                    self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                              K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        '''
        keras.backend.cast(x, dtype):
        '''
        if mask is not None:
            a *= K.cast(mask, K.floatx())

        '''
        keras.backend.epsilon():
        '''
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon()   , K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a

        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

"""Train the model"""

X_train, X_val, y_train, y_val,add_train,add_val = train_test_split(x_train_OE, ACP_y_train_,hc_train, test_size=0.2, random_state=1)

vocab_size = int(max(x_train_OE.max(), x_test_OE.max())) + 1
embedding_dim=100

(4748, 1)
                                                   0
0                              >1pos|ACP20mainTest|1
1  CETWRTETTGATGQASSLLSGRLLEQKAASCHNSYIVLCIENSFMT...
2                              >2pos|ACP20mainTest|1
3  DERCTIIIHPGSPCDPSDCVQYCYAEYNGVGKCIASKPGRSANCMC...
4                              >3pos|ACP20mainTest|1
(2690, 1)
                       0
0  >1pos|ACP20mainTest|1
1            FLWWLFKWAWK
2  >2pos|ACP20mainTest|1
3          FAKLAKKALAKLL
4  >3pos|ACP20mainTest|1
50
50
(2374, 745)


In [7]:
import json
import pandas as pd
from datetime import datetime

def append_result_row(csv_path, row_dict):
    df_row = pd.DataFrame([row_dict])
    # nếu chưa có file thì ghi header; nếu có rồi thì append không header
    if not os.path.exists(csv_path):
        df_row.to_csv(csv_path, index=False)
    else:
        df_row.to_csv(csv_path, index=False, mode="a", header=False)

In [8]:
import json
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


def rf_rank_features_on_train(X_train_feat, y_train, random_state=1):
    rf = RandomForestClassifier(
        n_estimators=500, n_jobs=-1,
        class_weight='balanced_subsample',
        random_state=random_state
    )
    rf.fit(X_train_feat, y_train)
    # trả về chỉ số cột sắp xếp theo độ quan trọng giảm dần
    return np.argsort(rf.feature_importances_)[::-1]

def lgbm_rank_features_on_train(X_train_feat, y_train, random_state=1):
    model = LGBMClassifier(num_leaves=32,n_estimators=888,max_depth=12,learning_rate=0.16,min_child_samples=50,random_state=2020,n_jobs=8)
    model.fit(X_train_feat, y_train)
    importantFeatures = model.feature_importances_
    # trả về chỉ số cột sắp xếp theo độ quan trọng giảm dần
    return np.argsort(importantFeatures)[::-1]

def CNN_SF(m, n, tag, save_dir="best_models", metrics_dir="metrics"):
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(metrics_dir, exist_ok=True)
    csv_path = os.path.join(metrics_dir, f"metrics_{tag}.csv")
    json_path = os.path.join(metrics_dir, f"metrics_{tag}.json")

    # === Rank 1 lần trên TRAIN ===
    if tag == 'RF':
        rank_idx = rf_rank_features_on_train(add_train, y_train, random_state=1)
    elif tag == 'LGBM':
        rank_idx = lgbm_rank_features_on_train(add_train, y_train, random_state=1)  # định nghĩa bên dưới
    else:
        raise ValueError("tag phải là 'RF' hoặc 'LGBM'")

    results = {}
    for i in range(m, n):
        print(f"Select {i} features")
        idx = rank_idx[:i]
        SF_ALL_K_train = add_train[:, idx]
        SF_ALL_K_val   = add_val[:,   idx]

        if 'hc_test' in globals():
            SF_ALL_K_test = hc_test[:, idx]
            sf_test_path = os.path.join(save_dir, f"SF_ALL_K_test_{tag}_k{i:03d}.npy")
            np.save(sf_test_path, SF_ALL_K_test)
            print(f"Saved SF_ALL_K_test to: {sf_test_path}")

        # === MODEL ===
        main_input = Input((maxlen,), dtype='int32', name='main_input')
        x = Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=True)(main_input)
        x = Conv1D(256, 3, activation='relu')(x)
        x_out = GlobalMaxPooling1D()(x)

        aux_input = Input((i,), name='aux_input')
        x = layers.concatenate([x_out, aux_input])
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.5)(x)
        main_output = Dense(1, activation='sigmoid', name='main_output')(x)

        model = Model(inputs=[main_input, aux_input], outputs=main_output)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        ckpt_path = os.path.join(save_dir, f"best_{tag}_k{i:03d}.keras")
        callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=ckpt_path, monitor='val_accuracy', mode='max',
                save_best_only=True, save_weights_only=False, verbose=0
            ),
            tf.keras.callbacks.EarlyStopping(
                monitor='val_accuracy', mode='max',
                patience=6, restore_best_weights=True, verbose=0
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_accuracy', mode='max',
                factor=0.5, patience=3, min_lr=1e-5, verbose=0
            )
        ]

        hist = model.fit(
            x={'main_input': X_train, 'aux_input': SF_ALL_K_train}, y=y_train,
            validation_data=({'main_input': X_val, 'aux_input': SF_ALL_K_val}, y_val),
            epochs=25, batch_size=64, verbose=0, callbacks=callbacks
        ).history

        best_val_acc = float(max(hist.get('val_accuracy', [0.0])))
        results[i] = {"best_val_acc": best_val_acc, "ckpt_path": ckpt_path}

        # Đo lại train/val acc ở best weights (đã restore)
        _, train_acc = model.evaluate(
            x={'main_input': X_train, 'aux_input': SF_ALL_K_train}, y=y_train, verbose=False
        )
        _, val_acc = model.evaluate(
            x={'main_input': X_val, 'aux_input': SF_ALL_K_val}, y=y_val, verbose=False
        )

        # Ghi 1 dòng vào CSV để theo dõi / so sánh
        append_result_row(csv_path, {
            "timestamp": datetime.now().isoformat(timespec="seconds"),
            "selector_tag": tag,
            "k_features": i,
            "train_acc": float(train_acc),
            "val_acc": float(val_acc),
            "best_val_acc": best_val_acc,
            "ckpt_path": ckpt_path
        })

        print(f"[k={i}] Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Best(val_acc): {best_val_acc:.4f}")
        print(f"Saved best model to: {ckpt_path}")

    # Lưu tổng hợp JSON
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"▶ Metrics CSV:  {csv_path}")
    print(f"▶ Metrics JSON: {json_path}")
    return results

CNN_SF(100,300,'RF')

Select 100 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_RF_k100.npy


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


[k=100] Train Acc: 0.9884 | Val Acc: 0.7916 | Best(val_acc): 0.7916
Saved best model to: best_models/best_RF_k100.keras
Select 101 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_RF_k101.npy
[k=101] Train Acc: 0.9200 | Val Acc: 0.7937 | Best(val_acc): 0.7937
Saved best model to: best_models/best_RF_k101.keras
Select 102 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_RF_k102.npy
[k=102] Train Acc: 0.9610 | Val Acc: 0.8000 | Best(val_acc): 0.8000
Saved best model to: best_models/best_RF_k102.keras
Select 103 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_RF_k103.npy
[k=103] Train Acc: 0.9105 | Val Acc: 0.7874 | Best(val_acc): 0.7874
Saved best model to: best_models/best_RF_k103.keras
Select 104 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_RF_k104.npy
[k=104] Train Acc: 0.9215 | Val Acc: 0.8042 | Best(val_acc): 0.8042
Saved best model to: best_models/best_RF_k104.keras
Select 105 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_RF

{100: {'best_val_acc': 0.7915789484977722,
  'ckpt_path': 'best_models/best_RF_k100.keras'},
 101: {'best_val_acc': 0.793684184551239,
  'ckpt_path': 'best_models/best_RF_k101.keras'},
 102: {'best_val_acc': 0.800000011920929,
  'ckpt_path': 'best_models/best_RF_k102.keras'},
 103: {'best_val_acc': 0.7873684167861938,
  'ckpt_path': 'best_models/best_RF_k103.keras'},
 104: {'best_val_acc': 0.8042105436325073,
  'ckpt_path': 'best_models/best_RF_k104.keras'},
 105: {'best_val_acc': 0.7873684167861938,
  'ckpt_path': 'best_models/best_RF_k105.keras'},
 106: {'best_val_acc': 0.800000011920929,
  'ckpt_path': 'best_models/best_RF_k106.keras'},
 107: {'best_val_acc': 0.7747368216514587,
  'ckpt_path': 'best_models/best_RF_k107.keras'},
 108: {'best_val_acc': 0.8021052479743958,
  'ckpt_path': 'best_models/best_RF_k108.keras'},
 109: {'best_val_acc': 0.7894737124443054,
  'ckpt_path': 'best_models/best_RF_k109.keras'},
 110: {'best_val_acc': 0.7915789484977722,
  'ckpt_path': 'best_models/be

In [13]:

CNN_SF(100,300,'LGBM')

[LightGBM] [Info] Number of positive: 963, number of negative: 936
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044446 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 31151
[LightGBM] [Info] Number of data points in the train set: 1899, number of used features: 607
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.507109 -> initscore=0.028438
[LightGBM] [Info] Start training from score 0.028438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


[k=100] Train Acc: 0.9194 | Val Acc: 0.7874 | Best(val_acc): 0.7874
Saved best model to: best_models/best_LGBM_k100.keras
Select 101 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_LGBM_k101.npy
[k=101] Train Acc: 0.8815 | Val Acc: 0.7874 | Best(val_acc): 0.7874
Saved best model to: best_models/best_LGBM_k101.keras
Select 102 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_LGBM_k102.npy
[k=102] Train Acc: 0.9974 | Val Acc: 0.7916 | Best(val_acc): 0.7916
Saved best model to: best_models/best_LGBM_k102.keras
Select 103 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_LGBM_k103.npy
[k=103] Train Acc: 0.9805 | Val Acc: 0.7916 | Best(val_acc): 0.7916
Saved best model to: best_models/best_LGBM_k103.keras
Select 104 features
Saved SF_ALL_K_test to: best_models/SF_ALL_K_test_LGBM_k104.npy
[k=104] Train Acc: 0.9537 | Val Acc: 0.8000 | Best(val_acc): 0.8000
Saved best model to: best_models/best_LGBM_k104.keras
Select 105 features
Saved SF_ALL_K_test to: best_model

{100: {'best_val_acc': 0.7873684167861938,
  'ckpt_path': 'best_models/best_LGBM_k100.keras'},
 101: {'best_val_acc': 0.7873684167861938,
  'ckpt_path': 'best_models/best_LGBM_k101.keras'},
 102: {'best_val_acc': 0.7915789484977722,
  'ckpt_path': 'best_models/best_LGBM_k102.keras'},
 103: {'best_val_acc': 0.7915789484977722,
  'ckpt_path': 'best_models/best_LGBM_k103.keras'},
 104: {'best_val_acc': 0.800000011920929,
  'ckpt_path': 'best_models/best_LGBM_k104.keras'},
 105: {'best_val_acc': 0.7915789484977722,
  'ckpt_path': 'best_models/best_LGBM_k105.keras'},
 106: {'best_val_acc': 0.7873684167861938,
  'ckpt_path': 'best_models/best_LGBM_k106.keras'},
 107: {'best_val_acc': 0.8063157796859741,
  'ckpt_path': 'best_models/best_LGBM_k107.keras'},
 108: {'best_val_acc': 0.7957894802093506,
  'ckpt_path': 'best_models/best_LGBM_k108.keras'},
 109: {'best_val_acc': 0.7915789484977722,
  'ckpt_path': 'best_models/best_LGBM_k109.keras'},
 110: {'best_val_acc': 0.793684184551239,
  'ckpt_p

In [4]:
# === CNN KHÔNG FEATURE SELECTION ===
import os, numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model

def CNN_noFS(save_dir="best_models_noFS"):
    os.makedirs(save_dir, exist_ok=True)

    feat_dim = add_train.shape[1]
    AUX_train = add_train
    AUX_val   = add_val

    if 'hc_test' in globals():
        AUX_test = hc_test
        sf_test_path = os.path.join(save_dir, f"SF_ALL_full_test_noFS.npy")
        np.save(sf_test_path, AUX_test)
        print(f"Saved full AUX test (no FS) to: {sf_test_path}")

    main_input = Input((maxlen,), dtype='int32', name='main_input')
    x = Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=True)(main_input)
    x = Conv1D(256, 3, activation='relu')(x)
    x_out = GlobalMaxPooling1D()(x)

    aux_input = Input((feat_dim,), name='aux_input')
    x = layers.concatenate([x_out, aux_input])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    main_output = Dense(1, activation='sigmoid', name='main_output')(x)

    model = Model(inputs=[main_input, aux_input], outputs=main_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    ckpt_path = os.path.join(save_dir, f"best_noFS_fullfeat.keras")
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=ckpt_path,
            monitor='val_accuracy', mode='max',
            save_best_only=True, save_weights_only=False, verbose=0
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', mode='max',
            patience=10, restore_best_weights=True, verbose=0
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_accuracy', mode='max',
            factor=0.5, patience=3, min_lr=1e-5, verbose=0
        )
    ]

    hist = model.fit(
        x={'main_input': X_train, 'aux_input': AUX_train}, y=y_train,
        validation_data=({'main_input': X_val, 'aux_input': AUX_val}, y_val),
        epochs=25, batch_size=64, verbose=1, callbacks=callbacks
    ).history

    best_val_acc = float(max(hist.get('val_accuracy', [0.0])))
    loss_tr, acc_tr = model.evaluate({'main_input': X_train, 'aux_input': AUX_train}, y_train, verbose=0)
    loss_va, acc_va = model.evaluate({'main_input': X_val,   'aux_input': AUX_val},   y_val,   verbose=0)

    print(f"[noFS | feat_dim={feat_dim}] Train Acc: {acc_tr:.4f} | Val Acc: {acc_va:.4f} | Best(val_acc): {best_val_acc:.4f}")
    print(f"Saved best model to: {ckpt_path}")

    return {
        "feat_dim": feat_dim,
        "best_val_acc": best_val_acc,
        "ckpt_path": ckpt_path
    }

# Gọi chạy
results_noFS = CNN_noFS()

Saved full AUX test (no FS) to: best_models_noFS/SF_ALL_full_test_noFS.npy
Epoch 1/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.6210 - loss: 0.6725 - val_accuracy: 0.7705 - val_loss: 0.6033 - learning_rate: 0.0010
Epoch 2/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.7514 - loss: 0.5769 - val_accuracy: 0.8000 - val_loss: 0.5013 - learning_rate: 0.0010
Epoch 3/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.8063 - loss: 0.4568 - val_accuracy: 0.7958 - val_loss: 0.4647 - learning_rate: 0.0010
Epoch 4/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.8799 - loss: 0.3394 - val_accuracy: 0.7937 - val_loss: 0.4633 - learning_rate: 0.0010
Epoch 5/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.9214 - loss: 0.2472 - val_accuracy: 0.7979 - val_loss: 0.4863 - learning_rate: 0.0010
Epoch 6/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9645 - loss: 0.1554 - val_accuracy: 0.8042 - val_loss: 0.5098 - learning_rate: 5.0000e-04
Epoch 7/25
30/3

In [5]:
# === CNN + ATTENTION POOLING ===
import os, numpy as np, tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, Dense, Dropout, Lambda
)
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

def temporal_attention_pooling(x, name_prefix="attn"):
    e = Dense(1, name=f"{name_prefix}_score")(x)
    a = Lambda(lambda t: tf.nn.softmax(t, axis=1), name=f"{name_prefix}_softmax")(e)
    context = Lambda(lambda t: K.sum(t[0] * t[1], axis=1),
                     name=f"{name_prefix}_context")([x, a])
    return context

def CNN_noFS_with_attention(save_dir="best_models_noFS_attn"):
    os.makedirs(save_dir, exist_ok=True)

    feat_dim = add_train.shape[1]
    AUX_train = add_train
    AUX_val   = add_val

    if 'hc_test' in globals():
        AUX_test = hc_test
        sf_test_path = os.path.join(save_dir, f"SF_ALL_full_test_noFS.npy")
        np.save(sf_test_path, AUX_test)
        print(f"Saved full AUX test (no FS) to: {sf_test_path}")

    main_input = Input((maxlen,), dtype='int32', name='main_input')
    emb = Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=True)(main_input)
    h = Conv1D(256, 3, activation='relu', name='conv')(emb)

    # Attention pooling thay cho GlobalMaxPooling1D
    x_out = temporal_attention_pooling(h, name_prefix="ta")

    aux_input = Input((feat_dim,), name='aux_input')
    x = layers.concatenate([x_out, aux_input], name='concat_main_aux')
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    main_output = Dense(1, activation='sigmoid', name='main_output')(x)

    model = Model(inputs=[main_input, aux_input], outputs=main_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    ckpt_path = os.path.join(save_dir, f"best_noFS_attn.keras")
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=ckpt_path, monitor='val_accuracy', mode='max',
            save_best_only=True, save_weights_only=False, verbose=0
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy', mode='max',
            patience=10, restore_best_weights=True, verbose=0
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_accuracy', mode='max',
            factor=0.5, patience=3, min_lr=1e-5, verbose=0
        )
    ]

    hist = model.fit(
        x={'main_input': X_train, 'aux_input': AUX_train}, y=y_train,
        validation_data=({'main_input': X_val, 'aux_input': AUX_val}, y_val),
        epochs=25, batch_size=64, verbose=1, callbacks=callbacks
    ).history

    best_val_acc = float(max(hist.get('val_accuracy', [0.0])))
    tr_loss, tr_acc = model.evaluate({'main_input': X_train, 'aux_input': AUX_train}, y_train, verbose=0)
    va_loss, va_acc = model.evaluate({'main_input': X_val,   'aux_input': AUX_val},   y_val,   verbose=0)

    print(f"[noFS+ATTN | feat_dim={feat_dim}] Train Acc: {tr_acc:.4f} | Val Acc: {va_acc:.4f} | Best(val_acc): {best_val_acc:.4f}")
    print(f"Saved best model to: {ckpt_path}")

    return {
        "feat_dim": feat_dim,
        "best_val_acc": best_val_acc,
        "ckpt_path": ckpt_path
    }

# Gọi chạy
results_noFS_attn = CNN_noFS_with_attention()


Saved full AUX test (no FS) to: best_models_noFS_attn/SF_ALL_full_test_noFS.npy
Epoch 1/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.5984 - loss: 0.6720 - val_accuracy: 0.7326 - val_loss: 0.6047 - learning_rate: 0.0010
Epoch 2/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.7469 - loss: 0.5725 - val_accuracy: 0.8021 - val_loss: 0.4786 - learning_rate: 0.0010
Epoch 3/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.8362 - loss: 0.4132 - val_accuracy: 0.7895 - val_loss: 0.4847 - learning_rate: 0.0010
Epoch 4/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8689 - loss: 0.3380 - val_accuracy: 0.7937 - val_loss: 0.5336 - learning_rate: 0.0010
Epoch 5/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.8869 - loss: 0.2900 - val_accuracy: 0.7832 - val_loss: 0.5520 - learning_rate: 0.0010
Epoch 6/25
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.9348 - loss: 0.2195 - val_accuracy: 0.7663 - val_loss: 0.6532 - learning_rate: 5.0000e-04
Epoch 7/25